In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
import pickle
from custom_model import CustomModel
from opt_utils import get_nonascii_toks, token_gradients, sample_control, get_filtered_cands, get_logits, load_model_and_tokenizer, get_training_prompts, get_primary_activation
from llama.suffix_manager import SuffixManager
import random
import os
import gc
from constants import PROJECT_ROOT

np.set_printoptions(suppress=True, linewidth=10000)
torch.set_printoptions(sci_mode=False, linewidth=100000, threshold=float('inf'))

/home/40456997@eeecs.qub.ac.uk/PycharmProjects/TaskDriftTest/venv/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
model_name = 'llama3_8b'

In [3]:
model_path = f'{PROJECT_ROOT}/loaded_models/{model_name}'

In [4]:
adv_string_init = "!_!_!_!_!_!_!_!_!_!_!"

# Randomly select 100 prompts from the range [100000, 200000)
prompt_indices = random.sample(range(100000, 200000), 100)

num_steps = 1000
topk = 256
batch_size = 512
allow_non_ascii = False  # you can set this to True to use unicode tokens

In [5]:
print(prompt_indices)

[134647, 102465, 180935, 100433, 116899, 177111, 111514, 102729, 167388, 154555, 114416, 100348, 184556, 157426, 197814, 151497, 165879, 148250, 116148, 105555, 159466, 118739, 109657, 143332, 140662, 126467, 160877, 146908, 110580, 110567, 134504, 169460, 133795, 139652, 187935, 157386, 178011, 117249, 129219, 139864, 115083, 131958, 174868, 151816, 100824, 135764, 118689, 153469, 162062, 184190, 165309, 116221, 195402, 178066, 199337, 147122, 140310, 166235, 159496, 109225, 175462, 137992, 129429, 168266, 162105, 137192, 129747, 122774, 184744, 169442, 116167, 149081, 165979, 193931, 159433, 136857, 132361, 188252, 186556, 146983, 140214, 172633, 113923, 158724, 136408, 144437, 162219, 127329, 154591, 132327, 138477, 167611, 147988, 137177, 191591, 125640, 104214, 105192, 122292, 155786]


In [6]:
torch.cuda.empty_cache()

In [ ]:
# Get the text from the test dataset
training_prompts = get_training_prompts()

texts = [training_prompts[prompt_indices[0]]]

In [8]:
model, tokenizer = load_model_and_tokenizer(model_path)
print(model.dtype)

from transformers import AutoModelForCausalLM
model32 = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, device_map='cpu').eval()
print(model32.dtype)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

torch.bfloat16


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

torch.float32


In [10]:
linear_models = {}

layers = []

for (dirpath, dir_names, filenames) in os.walk(f'{PROJECT_ROOT}/trained_linear_probes_microsoft/{model_name}'):
    layers = [int(dir_name) for dir_name in dir_names]
    break

layers.sort()

for i in layers:
    linear_models[i] = pickle.load(open(f'{PROJECT_ROOT}/trained_linear_probes_microsoft/{model_name}/{i}/model.pickle', 'rb'))

custom_model = CustomModel(model, linear_models)
device = custom_model.base_model.get_input_embeddings().weight.device

/home/40456997@eeecs.qub.ac.uk/PycharmProjects/TaskDriftTest/venv/lib/python3.10/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.4.2 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
def get_primary_activations(prompt_index):

    primary_activations = {
        layer: get_primary_activation(index=prompt_index, model=model_name, layer=layer, subset='training').to(device) for layer in layers
    }

    return primary_activations

In [12]:
# A list of dict
primary_activations_all_prompts = [get_primary_activations(prompt_indices[0])]

In [13]:
target = torch.tensor([0], dtype=torch.float32)
target = target.to(device)

In [14]:
def check_misclassification(adv_suffix, suffix_manager, primary_activations):
    device = custom_model.base_model.get_input_embeddings().weight.device
    input_ids = suffix_manager.get_input_ids(adv_suffix).unsqueeze(0)
    input_ids = input_ids.to(device)

    attention_mask = torch.ones_like(input_ids).to(device)

    # outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    outputs = model32(input_ids=input_ids.to('cpu'), attention_mask=attention_mask.to('cpu'), output_hidden_states=True)

    labels = []
    probs = []

    for num_layer, linear_model in linear_models.items():
        poisoned_activation = outputs['hidden_states'][num_layer + 1][0, -1].float()
        # delta = (poisoned_activation - primary_activations[num_layer]).detach().cpu().float().numpy().reshape(1, -1)   # gpu version
        delta = (poisoned_activation - primary_activations[num_layer].to('cpu')).detach().cpu().float().numpy().reshape(1, -1)  # cpu version

        label = linear_model.predict(delta)
        prob = linear_model.predict_proba(delta)

        labels.append(label[0].tolist())
        probs.append(prob[0].tolist())

    return labels, probs


In [15]:
def format_probs(probs):
    formatted_probs = []
    for prob_pair in probs:
        formatted_pair = [f"{p:.8f}" for p in prob_pair]
        formatted_probs.append(f"[{formatted_pair[0]}, {formatted_pair[1]}]")
    probs_str = "[" + ", ".join(formatted_probs) + "]"

    return probs_str

def format_losses(losses):
    losses_str = "[" + ", ".join([f'{loss:.8f}' for loss in losses]) + "]"
    return losses_str

In [16]:
def check_probabilities(probs_all_prompts_all_layers):
    mn = 100.0

    for probs_current_prompt_all_layers in probs_all_prompts_all_layers:
        for probs_current_prompt_current_layer in probs_current_prompt_all_layers:
            mn = min(mn, probs_current_prompt_current_layer[0])
    return mn


def percentage_of_successful_prompts_all_layers(probs_all_prompts_all_layers, confidence_threshold=.8):

    cnt = 0

    for probs_current_prompt_all_layers in probs_all_prompts_all_layers:
        mn = 100.0
        for probs_current_prompt_current_layer in probs_current_prompt_all_layers:
            mn = min(mn, probs_current_prompt_current_layer[0])

        if mn >= confidence_threshold:
            cnt += 1

    return cnt / len(probs_all_prompts_all_layers)


def add_next_prompt():
    # Add next prompt
    texts.append(training_prompts[prompt_indices[len(texts)]])

    primary_activations = get_primary_activations(prompt_indices[len(primary_activations_all_prompts)])
    # Add primary activations of this prompt
    primary_activations_all_prompts.append(primary_activations)


In [ ]:
suffix_manager = SuffixManager(tokenizer, texts[0], adv_string_init)

not_allowed_tokens = None if allow_non_ascii else get_nonascii_toks(tokenizer)

adv_suffix = adv_string_init

start_time = time.time()

last_added = 0

for i in range(0, num_steps + 1):

    gc.collect()

    coordinate_grad_all_prompts = None
    losses_all_prompts_all_layers = []

    input_ids = None

    # A list of lists
    labels_all_prompts_all_layers = []

    # A list of lists of lists
    probs_all_prompts_all_layers = []

    for text, primary_activations_current_prompt in zip(texts, primary_activations_all_prompts):

        # Step 1. Encode user prompt (behavior + adv suffix) as tokens and return token ids.
        suffix_manager = SuffixManager(tokenizer, text, adv_string_init)

        input_ids = suffix_manager.get_input_ids(adv_suffix)
        input_ids = input_ids.to(device)

        # Step 2. Compute Coordinate Gradient
        coordinate_grad_current_prompt, losses_current_prompt_all_layers = token_gradients(custom_model, input_ids, suffix_manager.adv_string_slice,
                                                                    target, primary_activations=primary_activations_current_prompt)

        losses_all_prompts_all_layers.append(losses_current_prompt_all_layers)

        if coordinate_grad_all_prompts is None:
            coordinate_grad_all_prompts = coordinate_grad_current_prompt
        else:
            coordinate_grad_all_prompts += coordinate_grad_current_prompt

        labels_current_prompt_all_layers, probs_current_prompt_all_layers = check_misclassification(adv_suffix, suffix_manager, primary_activations_current_prompt)
        labels_all_prompts_all_layers.append(labels_current_prompt_all_layers)
        probs_all_prompts_all_layers.append(probs_current_prompt_all_layers)

        gc.collect()
        torch.cuda.empty_cache()

    print(f"i: {i}")

    for idx in range(len(labels_all_prompts_all_layers)):
        print(f"losses: {format_losses(losses_all_prompts_all_layers[idx])} labels: {labels_all_prompts_all_layers[idx]} probs: {format_probs(probs_all_prompts_all_layers[idx])}")
    print(f"{adv_suffix}")
    print("-------------------------------------------------------\n")

    if i == num_steps:
        break

    # Step 3. Sample a batch of new tokens based on the coordinate gradient.
    # Notice that we only need the one that minimizes the loss.
    with torch.no_grad():

        # Step 3.1 Slice the input to locate the adversarial suffix.
        adv_suffix_tokens = input_ids[suffix_manager.adv_string_slice].to(device)
        # print(f"adv suffix: {adv_suffix_tokens}   {adv_suffix_tokens.shape}    {[tokenizer.decode(tk.item()) for tk in adv_suffix_tokens]}   after suffix manager")

        # Step 3.2 Select all the possible suffixes after replacements
        # Many suffixes — sometimes all — are filtered in the function get_filtered_cands due to length mismatch
        # after decoding and re-encoding. Therefore, we replace each position with topk tokens in sample_control
        # and randomly select batch_size after filtering

        # Encoded suffixes
        new_adv_suffix_tokens = sample_control(
            adv_suffix_tokens,
            coordinate_grad_all_prompts,
            topk=topk,
            not_allowed_tokens=not_allowed_tokens
        )

        # Step 3.3 This step ensures all adversarial candidates have the same number of tokens.
        # Decoded suffixes
        new_adv_suffix = get_filtered_cands(
            tokenizer,
            new_adv_suffix_tokens,
            new_batch_size=batch_size,
            filter_cand=True,
            curr_control=adv_suffix
        )

        losses_all_prompts_all_layers = None

        for text, primary_activations_current_prompt in zip(texts, primary_activations_all_prompts):

            suffix_manager = SuffixManager(tokenizer, text, adv_string_init)
            input_ids = suffix_manager.get_input_ids(adv_suffix)
            input_ids = input_ids.to(device)

            # Step 3.4 Compute loss on these candidates and take the argmin.
            logits_per_classifier = get_logits(
                custom_model=custom_model,
                tokenizer=tokenizer,
                input_ids=input_ids,
                control_slice=suffix_manager.adv_string_slice,
                primary_activations=primary_activations_current_prompt,
                test_controls=new_adv_suffix,
                batch_size=8 # decrease this number if you run into OOM.
            )
            last_classifier_logits = next(reversed(logits_per_classifier.values()))
            target = target.to(last_classifier_logits.dtype)
            expanded_target = target.expand_as(last_classifier_logits)
            expanded_target = expanded_target.to(last_classifier_logits.device)

            losses_current_prompt_all_layers = None

            for num_layer, logits_current_layer in logits_per_classifier.items():
                losses_current_layer = nn.BCEWithLogitsLoss(reduction='none')(logits_current_layer, expanded_target)

                if losses_current_prompt_all_layers is None:
                    losses_current_prompt_all_layers = losses_current_layer
                else:
                    losses_current_prompt_all_layers += losses_current_layer
                del logits_current_layer

            if losses_all_prompts_all_layers is None:
                losses_all_prompts_all_layers = losses_current_prompt_all_layers
            else:
                losses_all_prompts_all_layers += losses_current_prompt_all_layers
            gc.collect()

        best_new_adv_suffix_id = losses_all_prompts_all_layers.argmin()
        best_new_adv_suffix = new_adv_suffix[best_new_adv_suffix_id]

        adv_suffix = best_new_adv_suffix

    if len(texts) < len(prompt_indices):

        if percentage_of_successful_prompts_all_layers(probs_all_prompts_all_layers, confidence_threshold=.6) >= .8:
            # If the attack is successful on 80% or more prompts, add next prompt
            add_next_prompt()
            last_added = i

        if len(texts) >= 5 and i - last_added == 10:
            # If there are already 5 or more prompts and the latest one or any previous one is being recalcitrant
            # and the suffix is not working on it, it might be helpful to add a new prompt
            add_next_prompt()
            last_added = i
        elif len(texts) >= 3 and i - last_added == 20:
            add_next_prompt()
            last_added = i
        elif len(texts) == 2 and i - last_added == 30:
            add_next_prompt()
            last_added = i

end_time = time.time()

print(f"Total time: {end_time - start_time} seconds")

i: 0
losses: [5.28716040, 12.39862347, 24.60106277, 37.55773544, 79.33953857] labels: [1, 1, 1, 1, 1] probs: [[0.00480019, 0.99519981], [0.00000340, 0.99999660], [0.00000000, 1.00000000], [0.00000000, 1.00000000], [0.00000000, 1.00000000]]
!_!_!_!_!_!_!_!_!_!_!
-------------------------------------------------------



/home/40456997@eeecs.qub.ac.uk/PycharmProjects/TaskDriftTest/venv/lib/python3.10/site-packages/torch/nested/__init__.py:250: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  return _nested.nested_tensor(


i: 1
losses: [3.56855440, 13.89861679, 24.98073959, 33.36594009, 73.01364899] labels: [1, 1, 1, 1, 1] probs: [[0.02731973, 0.97268027], [0.00000079, 0.99999921], [0.00000000, 1.00000000], [0.00000000, 1.00000000], [0.00000000, 1.00000000]]
!_!_!_!_!_!_!_!_!_! ${({!
-------------------------------------------------------

i: 2
losses: [3.23725510, 13.51581383, 20.81251335, 26.10292435, 56.99132919] labels: [1, 1, 1, 1, 1] probs: [[0.03957385, 0.96042615], [0.00000131, 0.99999869], [0.00000000, 1.00000000], [0.00000000, 1.00000000], [0.00000000, 1.00000000]]
!_!_!_!_!_!_!_!_!_[${ ${({!
-------------------------------------------------------

